In [1]:
import numpy as np
import pandas as pd

# Add manufacturer to iqvia data

In [2]:
df_IQVIA_Data = pd.read_excel('IQVIA Additional Drug Detail v72020.xlsx')
df_IQVIA_Data['NDC'] = [e[0] for e in df_IQVIA_Data.NDC.str.split(' ')]
df_IQVIA_Data.NDC.astype(str).str.zfill(11)

0        00002060440
1        00002105202
2        00002143301
3        00002143380
4        00002143401
            ...     
12664    99207051114
12665    99207051117
12666    99207051413
12667    99207051746
12668    99207085060
Name: NDC, Length: 12669, dtype: object

In [3]:
df_IQVIA_Data.NDC = df_IQVIA_Data.NDC.apply(lambda x:int(x))

In [4]:
temp = pd.read_csv('merge_with_iqvia_2017_01_2020_09_monthly.csv')

In [5]:
temp = temp[['NDC','Manufacturer']]

In [6]:
df_IQVIA_Data = pd.merge(df_IQVIA_Data,temp,left_on='NDC',right_on='NDC',how='inner')

# Concat all WAC data

In [8]:
df = pd.read_excel('2014-2016 WAC History.xlsx')
df = pd.concat([pd.DataFrame({'Date': pd.date_range(row.EffectiveDate, row.EndDate, freq='M'),
               'WAC': row.Price,
               'Drug Identifier': row.Identifier}, columns=['Date', 'WAC', 'Drug Identifier']) 
           for i, row in df.iterrows()], ignore_index=True)

In [9]:
df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.apply(lambda x:x.month)
df['Year'] = df.Date.apply(lambda x:x.year)
df.drop('Date',axis=1,inplace = True)

In [10]:
df.rename(columns={"Drug Identifier": "NDC"},inplace = True)

In [11]:
df1 = pd.read_csv('merge_with_iqvia_2017_01_2020_09_monthly.csv')

In [12]:
df1= df1[['NDC','WAC','Month','Year']]

In [13]:
df = pd.concat([df,df1],sort = True)

In [14]:
df.drop_duplicates(keep = 'first',inplace = True)

In [25]:
df.to_csv('monthly_data_27k_drugs.csv')

# Merge with iqivia

In [19]:
df_IQVIA_Data.drop_duplicates(keep = 'first',inplace = True)

In [8]:
temp = df_IQVIA_Data.NDC.unique().tolist()
df = df[df.NDC.isin(temp)]

In [20]:
merge = pd.merge(df,df_IQVIA_Data,left_on='NDC',right_on='NDC',how='inner')

# Pivot TRx

In [40]:
columns = ['Aug 2014\nTRx', 'Sep 2014\nTRx', 'Oct 2014\nTRx',
       'Nov 2014\nTRx', 'Dec 2014\nTRx', 'Jan 2015\nTRx', 'Feb 2015\nTRx',
       'Mar 2015\nTRx', 'Apr 2015\nTRx', 'May 2015\nTRx', 'Jun 2015\nTRx',
       'Jul 2015\nTRx', 'Aug 2015\nTRx', 'Sep 2015\nTRx', 'Oct 2015\nTRx',
       'Nov 2015\nTRx', 'Dec 2015\nTRx', 'Jan 2016\nTRx', 'Feb 2016\nTRx',
       'Mar 2016\nTRx', 'Apr 2016\nTRx', 'May 2016\nTRx', 'Jun 2016\nTRx',
       'Jul 2016\nTRx', 'Aug 2016\nTRx', 'Sep 2016\nTRx', 'Oct 2016\nTRx',
       'Nov 2016\nTRx', 'Dec 2016\nTRx', 'Jan 2017\nTRx', 'Feb 2017\nTRx',
       'Mar 2017\nTRx', 'Apr 2017\nTRx', 'May 2017\nTRx', 'Jun 2017\nTRx',
       'Jul 2017\nTRx', 'Aug 2017\nTRx', 'Sep 2017\nTRx', 'Oct 2017\nTRx',
       'Nov 2017\nTRx', 'Dec 2017\nTRx', 'Jan 2018\nTRx', 'Feb 2018\nTRx',
       'Mar 2018\nTRx', 'Apr 2018\nTRx', 'May 2018\nTRx', 'Jun 2018\nTRx',
       'Jul 2018\nTRx', 'Aug 2018\nTRx', 'Sep 2018\nTRx', 'Oct 2018\nTRx',
       'Nov 2018\nTRx', 'Dec 2018\nTRx', 'Jan 2019\nTRx', 'Feb 2019\nTRx',
       'Mar 2019\nTRx', 'Apr 2019\nTRx', 'May 2019\nTRx', 'Jun 2019\nTRx',
       'Jul 2019\nTRx', 'Aug 2019\nTRx', 'Sep 2019\nTRx', 'Oct 2019\nTRx',
       'Nov 2019\nTRx', 'Dec 2019\nTRx', 'Jan 2020\nTRx', 'Feb 2020\nTRx',
       'Mar 2020\nTRx', 'Apr 2020\nTRx', 'May 2020\nTRx', 'Jun 2020\nTRx',
       'Jul 2020\nTRx']

In [41]:
test = merge.melt(id_vars=[ 'NDC','Year','Month'],  value_vars=columns)
test.value.fillna(0,inplace = True)
dict_={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
def fun(year,month,column):
    if dict_[month] == column[:3] and year == int(column[4:8]):
        return True
    else:
        return False
test['a'] = test.apply(lambda x:fun(x.Year,x.Month,x.variable),axis = 1)
test = test[test.a == True]
test.drop(['a','variable'],axis = 1,inplace=True)
test.rename(columns={"value": "TRx"},inplace = True)
merge =  pd.merge(merge, test,  how='inner', left_on=['NDC','Year','Month'], right_on = ['NDC','Year','Month'])
merge = merge.loc[:, ~merge.columns. isin(columns)]

In [50]:
merge['TRx'].replace(0, np.nan,inplace = True)

In [52]:
merge.to_csv('monthly_2014_8_2020_7_4k_drugs.csv')